In [124]:
#import required modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

%matplotlib inline

In [125]:
#unpack pickled dataframes

modeling = pd.read_pickle("modeling.pkl")
forecast = pd.read_pickle("forecast.pkl")

#change differential into win/loss
modeling["Differential"] = modeling["Differential"].apply(lambda x: 1 if x > 0 else 0)
modeling.head()

,Differential,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Chiefs Texans 2020,1,90.2,84.5,86.1,85.0,87.6,71.2,71.9,83.1,75.4,...,81.5,75.1,73.4,68.6,76.7,59.4,87.0,67.3,59.1,53.4
Seahawks Falcons 2020,1,92.0,81.9,79.6,77.9,68.4,74.1,87.6,90.5,73.4,...,90.1,79.7,76.6,76.6,72.0,53.0,72.0,72.1,87.7,66.6
Bills Jets 2020,1,64.1,78.3,60.2,91.3,65.1,73.1,74.3,65.6,77.7,...,81.3,83.0,79.0,73.9,87.9,60.6,73.1,72.0,56.1,65.6
Raiders Panthers 2020,1,79.9,75.5,83.2,87.1,73.1,64.9,67.4,79.0,77.9,...,90.4,79.3,63.8,65.9,86.7,76.2,84.7,63.4,58.6,63.6
Bears Lions 2020,1,76.6,80.8,58.0,66.6,91.7,58.6,83.8,61.9,66.8,...,85.9,75.5,82.8,72.3,74.4,74.4,83.0,74.9,64.9,78.7


In [126]:
#split modeling data in to test and train (75/25)
X = modeling.drop("Differential", axis = 1)
Y = modeling["Differential"]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 10)

In [127]:
#a kernelized SVM gives nice results with both training and test data
from sklearn.svm import SVC

SVM = SVC().fit(X_train,Y_train)
print(np.sum(SVM.predict(X_test) == np.array(Y_test))/len(X_test))
print(np.sum(SVM.predict(X_train) == np.array(Y_train))/len(X_train))

0.6477272727272727
0.7929292929292929


In [128]:
#cross validation of the SVM model
#on average the model will predict the game result correctly ~63% of the time
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(SVM, X, Y, cv = 10)
print(np.average(cv_scores))

0.6259299191374664


In [129]:
#testing out a dummy classifier to validate model
#we can see a 13% increase in accuracy with the NB model
from sklearn.dummy import DummyClassifier

dummy_majority = DummyClassifier(strategy = "most_frequent").fit(X_train,Y_train)
cv_scores = cross_val_score(dummy_majority,X,Y,cv = 10)
print(np.average(cv_scores))

0.5019047619047619


In [130]:
#applying the model to forecast the 2022 season
SVM = SVC().fit(X,Y)
forecast["result"] = SVM.predict(forecast)

In [131]:
#relabel and export to csv
forecast["result"] = forecast["result"].apply(lambda x: "W" if x == 1 else "L")
forecast["result"].to_frame().to_csv("2022predictions.csv")